In [1]:
# https://developer.musixmatch.com/documentation/api-reference/track-get

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import os
import sys
import re
import time
import requests
%matplotlib inline

from dotenv import load_dotenv
%load_ext dotenv
%dotenv

# Example: Use MusicMatch API to find lyrics

In [3]:
# https://developer.musixmatch.com/documentation/api-reference/track-get


base_url = 'https://api.musixmatch.com/ws/1.1/'
api_key = os.environ['MUSIXMATCH_API_KEY']

isrc = 'AUAB00000088'

query_string = f"{base_url}matcher.lyrics.get?track_isrc={isrc}&apikey={api_key}"
response = requests.get(query_string)


with open('outputfile.json', 'wb') as outf:
    outf.write(response.content)

response = response.json()
lyrics = response['message']['body']['lyrics']['lyrics_body']
explicit = response['message']['body']['lyrics']['explicit']

print (lyrics)

Greg: Well, it's Christmas time in Wiggle World
And as you can see
We're dancing round and decorating
Our Wiggly Christmas tree
We'll have lots of fun
In our Christmas way
We just can't wait for Christmas day
Wiggles: Can't wait for Christmas day, can't wait for Christmas day
Well, Murray and Greg are having fun
Hanging the Christmas star
Murray: This is fun
Greg: You'll hear Wags the Dog bark his Christmas song
No matter where you are
...

******* This Lyrics is NOT for Commercial use *******
(1409621371225)


# Load Tracks data

In [4]:
tracks_df = pd.read_csv("../data/track_isrc.csv")
tracks_df.head(3)

,Track_ID,ISRC,Track_Name,Artists,popularity,preview_url
0,2FPQI1LRwWszttbRG8hknk,USSM19601624,Games Monsters Play,"['Herry Monster', 'Grover']",6,https://p.scdn.co/mp3-preview/33cc59cc1836954e...
1,6pOoswwC1lNBI2TapMdaEW,USSM19600385,Afraid of the Dark,['Telly Monster'],6,https://p.scdn.co/mp3-preview/cf340f0b536edadd...
2,2EEwSq98rKwlRWT7sNCLRc,USSM19700463,Eensy Weensy Spider,"['Count Von Count', 'The Sesame Street Kids']",10,https://p.scdn.co/mp3-preview/4cdc12aaeb7da4b7...


In [5]:
# Check null values

print (len(tracks_df))
tracks_df.isnull().sum() 

# All tracks have ISRC.  

18074


Track_ID          0
ISRC              0
Track_Name        0
Artists           0
popularity        0
preview_url    7790
dtype: int64

In [6]:
base_url = 'https://api.musixmatch.com/ws/1.1/'
api_key = os.environ['MUSIXMATCH_API_KEY']

# Feb 10, 10AM   idx 1800: 3700
# Feb 11, 10:30AM  index 3700:5600
# Feb 11, 10:50AM, index 5600:5700
# Feb 11, 10:52AM  index 5700:6000
# Feb 12,  12:40PM index 6000:8000
# Feb 12,  4:20PM index 8000:10000
# Feb 13, 11:30AM  index 10000:12000
# Feb 13, 13:40PM  index 12000:14000
# Feb 14, 13:00PM index 14000:16000
# Feb 14, 23:10PM index 16000:18000
# Feb 15, 23:20PM index 16000:18000   (try again to compare with yesterday)
# Feb 15, 23:34 PM index 18000:18074


for (i, row) in tracks_df[18000:].iterrows():
        
    isrc = row['ISRC']
    query_string = f"{base_url}matcher.lyrics.get?track_isrc={isrc}&apikey={api_key}"
    response = requests.get(query_string)
    
    filepath = os.path.join('../data/lyrics/',isrc + ".json")

    with open(filepath, 'wb') as outf:
        outf.write(response.content)
        
    if i%100 == 0:
        time.sleep(3)


In [4]:
import json

lyrics_df = []
for (i, row) in tracks_df.iterrows():
        
    isrc = row['ISRC']

    filepath = os.path.join('../data/lyrics/',isrc + ".json")

    with open(filepath, 'r') as f:
        data = json.load(f)
        body = data['message']['body']
        if body:
            lyrics = body.get('lyrics')['lyrics_body']
            if lyrics:
                count += 1
                lyrics_df.append([isrc, lyrics])


In [6]:
#lyrics_df = pd.DataFrame(lyrics_df, columns = ['ISRC','lyrics'])
#lyrics_df.to_csv("../data/lyrics.csv", index = False)

lyrics_df = pd.read_csv("../data/lyrics.csv")
display(lyrics_df.sample(3))
print ("Number of lyrics: ", lyrics_df.shape[0])

,ISRC,lyrics
5502,USMC10400184,You must not have a heart\nHave nothing in you...
6210,USUM70845900,If you feel like runnin' today\nYou know I'd u...
2308,GBAAA9600217,"(Ah, yeah, hey, yea, yeah)\n(Oh yeah)\nWe've g..."


Number of lyrics:  12900


In [7]:
print ("Feb 10, 1326 out of 3700 songs have lyrics \t", f"{1326/3700:.2f}")
print ("Feb 11, 3367 out of 6000 songs have lyrics \t", f"{3367/6000:.2f}")
nsongs, nlyrics = 8000, 5135 
print (f"Feb 12, {nlyrics} out of {nsongs} songs have lyrics \t {nlyrics/nsongs:.2f}")

nsongs, nlyrics = 10000, 6919 
print (f"Feb 12, {nlyrics} out of {nsongs} songs have lyrics \t {nlyrics/nsongs:.2f}")

nsongs, nlyrics = 14000, 10459
print (f"Feb 13, {nlyrics} out of {nsongs} songs have lyrics \t {nlyrics/nsongs:.2f}")

nsongs, nlyrics = 16000, 11782
print (f"Feb 14, {nlyrics} out of {nsongs} songs have lyrics \t {nlyrics/nsongs:.2f}")

nsongs, nlyrics = 18000, 12834
print (f"Feb 14, {nlyrics} out of {nsongs} songs have lyrics \t {nlyrics/nsongs:.2f}")

nsongs, nlyrics = 18074, 12900
print (f"Feb 15, {nlyrics} out of {nsongs} songs have lyrics \t {nlyrics/nsongs:.2f}")

Feb 10, 1326 out of 3700 songs have lyrics 	 0.36
Feb 11, 3367 out of 6000 songs have lyrics 	 0.56
Feb 12, 5135 out of 8000 songs have lyrics 	 0.64
Feb 12, 6919 out of 10000 songs have lyrics 	 0.69
Feb 13, 10459 out of 14000 songs have lyrics 	 0.75
Feb 14, 11782 out of 16000 songs have lyrics 	 0.74
Feb 14, 12834 out of 18000 songs have lyrics 	 0.71
Feb 15, 12900 out of 18074 songs have lyrics 	 0.71
